In [ ]:
## Visualization of Raw Data and Synthetic Data
import numpy as np
import matplotlib as mpl
import re
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import random
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib
import matplotlib.colors as mcolors
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import src.vae.api as vpi

import torch

import mlflow
from config.loader import load_config
import os

import src.measurements.api as mpi
import src.generator.api as gpi
import src.peaks.api as ppi
import src.statistics.api as spi

plt.rcParams['text.usetex'] = True

In [ ]:
dates = mpi.API().unique_dates()

In [ ]:
# Load Real Measuremnt Data
dates = mpi.API().unique_dates()
# measurement = mpi.API().measurement(dates=dates)
processed_measurements = ppi.API().measurement(dates=dates)

In [ ]:
synthetic_keys = vpi.API().unique_dates()
random.shuffle(synthetic_keys)
synthetics = vpi.API().re_synhtetics(dates=synthetic_keys[0:1246])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.rcParams['text.usetex'] = True

# Schwellenwerte (10^1 bis 10^6)
thresholds = [10**i for i in range(1, 7)]

# Anzahl eindeutiger Messungen pro Schwelle
counts_unique_meas = []
for t in thresholds:
    counted = synthetics.loc[
        synthetics["count"] > t
    ].drop_duplicates(subset=["datetime"])
    counts_unique_meas.append(len(counted))

# Prozentwerte berechnen
total = counts_unique_meas[0] if counts_unique_meas else 1
percent_labels = [f"{c}\n({100 * c / total:.1f}\%)" for c in counts_unique_meas]

# Achsenbeschriftungen (LaTeX für 10^x)
labels = [f"$>10^{int(np.log10(t))}$" for t in thresholds]


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
bars = axs[0].bar(labels, counts_unique_meas, color="white", edgecolor="black")

# Prozent über die Balken
axs[0].bar_label(bars, labels=percent_labels, fontsize=12)

# Achsentitel
axs[0].set_xlabel("Schwellenwert (Zählwerte)", fontsize=14, labelpad=10)
axs[0].set_ylabel("Anzahl der Messungen", fontsize=14, labelpad=10)
axs[0].grid(False)
axs[0].tick_params(axis='x', labelsize=12, bottom=True)
axs[0].tick_params(axis='y', labelsize=12, left=True)
axs[0].set_ylim(0, 1400)

y = sns.histplot(
    abs(synthetics["count"].to_numpy() + 1),
    bins=40,
    stat="probability",
    color="black",
    alpha=0.6,
    label="Data",
    log_scale=(True, False),
    ax=axs[1],
)
axs[1].set_xlabel("Log(Zählwert + 1)", labelpad=10, size=14)
axs[1].set_ylabel("Wahrscheinlichkeit", labelpad=10, size=14)
axs[1].grid(True, alpha=0.3, which="both")
axs[1].set_xlim(
    1,
)

axs[1].tick_params(axis='x', labelsize=12, bottom=True)
axs[1].tick_params(axis='y', labelsize=12, left=True)
fig.tight_layout(pad=2)
fig.savefig("plots\\percentage_histogram_measurement_with_threshold_synthetic.pdf")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
plt.rcParams['text.usetex'] = True

# Schwellenwerte (10^1 bis 10^6)
thresholds = [10**i for i in range(1, 7)]

# Anzahl eindeutiger Messungen pro Schwelle
counts_unique_meas = []
for t in thresholds:
    counted = processed_measurements.loc[
        processed_measurements["count"] > t
    ].drop_duplicates(subset=["datetime"])
    counts_unique_meas.append(len(counted))

# Prozentwerte berechnen
total = counts_unique_meas[0] if counts_unique_meas else 1
percent_labels = [f"{c}\n({100 * c / total:.1f}\%)" for c in counts_unique_meas]

# Achsenbeschriftungen (LaTeX für 10^x)
labels = [f"$>10^{int(np.log10(t))}$" for t in thresholds]


fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))
bars = axs[0].bar(labels, counts_unique_meas, color="white", edgecolor="black")

# Prozent über die Balken
axs[0].bar_label(bars, labels=percent_labels, fontsize=12)

# Achsentitel
axs[0].set_xlabel("Schwellenwert (Zählwerte)", fontsize=14, labelpad=10)
axs[0].set_ylabel("Anzahl der Messungen", fontsize=14, labelpad=10)
axs[0].grid(False)
axs[0].tick_params(axis='x', labelsize=12, bottom=True)
axs[0].tick_params(axis='y', labelsize=12, left=True)
axs[0].set_ylim(0, 1400)

y = sns.histplot(
    abs(processed_measurements["count"].to_numpy() + 1),
    bins=40,
    stat="probability",
    color="black",
    alpha=0.6,
    label="Data",
    log_scale=(True, False),
    ax=axs[1],
)
axs[1].set_xlabel("Log(Zählwert + 1)", labelpad=10, size=14)
axs[1].set_ylabel("Wahrscheinlichkeit", labelpad=10, size=14)
axs[1].grid(True, alpha=0.3, which="both")
axs[1].set_xlim(
    1,
)

axs[1].tick_params(axis='x', labelsize=12, bottom=True)
axs[1].tick_params(axis='y', labelsize=12, left=True)
fig.tight_layout(pad=2)
fig.savefig("plots\\percentage_histogram_measurement_with_threshold.pdf")
plt.show()


In [ ]:
dates = mpi.API().unique_dates()
measurement = mpi.API().measurement(dates)

In [ ]:
meas_filtered = measurement.loc[measurement["energy"] < 0].reset_index(drop=True)
meas_filtered

In [ ]:
measurement_diffs = (
    measurement.sort_values(by="energy").groupby("datetime").diff().dropna()
)
measurement_diffs["energy"] = measurement_diffs["energy"].round(2)
measurement_diffs = measurement_diffs.join(
    measurement, lsuffix="_diffs", rsuffix="_raw", how="left"
)
diffs = measurement_diffs.groupby("energy_diffs").count().reset_index()
diffs["percent"] = diffs["count_diffs"] / diffs["count_diffs"].sum() * 100
measurement_diffs.describe().apply(lambda s: s.apply("{0:.5f}".format))

mean = measurement_diffs["energy_diffs"].mean()
plt.rcParams['text.usetex'] = True
data = measurement.groupby("datetime")["energy"].max().reset_index()
data["energy"] = data["energy"].round(0).astype(int)
data = data.groupby("energy").count().reset_index()

data["percent"] = data["datetime"] / data["datetime"].sum() * 100

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

ax = sns.barplot(
    diffs,
    x="energy_diffs",
    y="count_diffs",
    color="grey",
    linewidth=1.5,
    edgecolor=".5",
    facecolor=(0, 0, 0, 0),
    ax=axs[0],
)
labels = [
    f"{c / 1e3:.1f}K\n({p:.2f}\%)"
    for c, p in zip(diffs["count_diffs"], diffs["percent"])
]
axs[0].bar_label(ax.containers[0], labels=labels, fontsize=12)
axs[0].vlines(x=mean+2.15, ymin=1, ymax=100000000, color="black", linestyle="dotted", label="$\mu=0.34507$")
# plt.title("Untersuchung der Energiewerte")
axs[0].set_xlabel("Energiedifferenz [keV]", size=14, labelpad=10)
axs[0].set_ylabel("Log(Anzahl)", size=14, labelpad=10)
axs[0].tick_params(axis='x', labelsize=12, bottom=True)
axs[0].tick_params(axis='y', labelsize=12, left=True)
axs[0].grid(False)
axs[0].legend()
axs[0].set_yscale("log")
axs[0].set_ylim(1, 100000000)


ax2 = sns.barplot(
    data,
    x="energy",
    y="datetime",
    color="grey",
    linewidth=1.5,
    edgecolor=".5",
    facecolor=(0, 0, 0, 0),
    ax=axs[1],
)
labels = [
    f"{c / 1e0:.0f}\n({p:.2f}\%)"
    for c, p in zip(data["datetime"], data["percent"])
]
axs[1].bar_label(axs[1].containers[0], labels=labels, fontsize=12)
# plt.title("Untersuchung der Energiewerte")
axs[1].set_xlabel("Max(Energie [keV])", size=14, labelpad=10)
axs[1].set_ylabel("Log(Anzahl)", size=14, labelpad=10)
axs[1].tick_params(axis='x', labelsize=12, bottom=True)
axs[1].tick_params(axis='y', labelsize=12, left=True)
axs[1].grid(False)
axs[1].set_yscale("log")
axs[1].set_ylim(1, 10000)
fig.tight_layout(pad=2)
fig.savefig("plots\\energy_diffs2.pdf")
plt.show()
plt.close()

In [ ]:
test = measurement.loc[measurement["datetime"] == measurement["datetime"].unique()[15]].reset_index(drop=True)

test2 = synthetics.loc[synthetics["datetime"] == synthetics["datetime"].unique()[15]].reset_index(drop=True)

In [ ]:
sns.histplot(
    abs(test2["count"].to_numpy() + 1),
    bins=40,
    stat="probability",
    color="black",
    alpha=0.6,
    label="Data",
    log_scale=(True, False),
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
#

# Raw or log-transformed data
counts = np.log(synthetics["count"].to_numpy() +1)
counts_mess = np.log(processed_measurements["count"].to_numpy() + 1)

stats.probplot(counts, dist='expon', sparams=(np.mean(counts),), plot=plt)
stats.probplot(counts_mess, dist='expon', sparams=(np.mean(counts_mess),), plot=plt)
# plt.xticks([0, 5, 10, 15])
# plt.ylim(0,1000)
plt.show()


In [ ]:
pca_df

In [ ]:
synthetics

In [ ]:
pivot_df_synthetics

In [ ]:
pivot_df

In [ ]:
# Filter for rows where peak is True
df_peaks = re_processed_measurements[re_processed_measurements["peak"] == True]

# Group by datetime, and join identified_isotopes into a comma-separated string
merged_isotopes = df_peaks.groupby("datetime")["identified_isotope"] \
    .apply(lambda x: ', '.join(sorted(set(x)))) \
    .reset_index()

# Rename column
merged_isotopes.columns = ["datetime", "identified_isotope"]

# Filter for rows where peak is True
df_peaks_synthetics = synthetics[synthetics["peak"] == True]

# Group by datetime, and join identified_isotopes into a comma-separated string
merged_isotopes_synthetics = df_peaks_synthetics.groupby("datetime")["identified_isotope"] \
    .apply(lambda x: ', '.join(sorted(set(x)))) \
    .reset_index()

# Rename column
merged_isotopes_synthetics.columns = ["datetime", "identified_isotope"]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Pivot to get a matrix: rows = samples, cols = energy bins
pivot_df = re_processed_measurements.pivot_table(
    index='datetime',  # each sample
    columns='energy',       # one column per energy
    values='count',         # values to use
    fill_value=0            # fill missing with 0
)
data_to_join = merged_isotopes.set_index("datetime")["identified_isotope"]

data_to_join_synthetics = merged_isotopes_synthetics.set_index("datetime")["identified_isotope"]

pivot_df_synthetics = synthetics.pivot_table(
    index='datetime',  # each sample
    columns='energy',       # one column per energy
    values='count',         # values to use
    fill_value=0            # fill missing with 0
)

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(pivot_df)

scaler_synthetics = StandardScaler()
scaled_data_synthetics = scaler_synthetics.fit_transform(pivot_df_synthetics)

# Run PCA
pca = PCA(n_components=2)
components = pca.fit_transform(scaled_data)

pca_synthetics = PCA(n_components=2)
compoents_synthetics = pca_synthetics.fit_transform(scaled_data_synthetics)

# Put results into DataFrame
pca_df = pd.DataFrame(components, columns=['PC1', 'PC2'])
pca_df['sample_number'] = pivot_df.index
pca_df["data"] = "real"

pca_df = pca_df.set_index("sample_number").join(data_to_join).reset_index()

pca_df_synthetics = pd.DataFrame(compoents_synthetics, columns=['PC1', 'PC2'])
pca_df_synthetics['sample_number'] = pivot_df_synthetics.index
pca_df_synthetics["data"] = "synthetics"

X = pca_df[["PC1", "PC2"]].to_numpy()
kmeans = KMeans(n_clusters=11, random_state=42)
pca_df["cluster"] = kmeans.fit_predict(X)

pca_df_synthetics = pca_df_synthetics.set_index("sample_number").join(data_to_join_synthetics).reset_index()

X = pca_df_synthetics[["PC1", "PC2"]].to_numpy()
kmeans = KMeans(n_clusters=11, random_state=42)
pca_df_synthetics["cluster"] = kmeans.fit_predict(X)

data = pd.concat([pca_df, pca_df_synthetics], axis=0)
data = data.sort_values(by="identified_isotope", ascending=False)

data


In [ ]:
filter_for_isos = data["identified_isotope"].unique()[3:20]

In [ ]:
# Plot
filtered_data = data.loc[data["identified_isotope"].isin(filter_for_isos)]
plt.figure(figsize=(6, 4))
data["cluster"] = data["cluster"].astype(str)
sns.relplot(data=data, x='PC1', y='PC2', kind="scatter", hue="identified_isotope",
            col="data", row="cluster", alpha=0.5, legend=False)
plt.title("PCA of Counts per Sample")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
plt.legend(title="Sample", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlim(0,100)
plt.ylim(-100,100)
plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


sns.kdeplot(counts, label="Real", fill=True, color="blue", alpha=0.4)
sns.kdeplot(counts_mess, label="Synthetic", fill=True, color="orange", alpha=0.4)
plt.legend()
plt.title("Density Comparison")
plt.show()

In [ ]:
dates[100:]

In [ ]:
def __identify_background(data, wndw: int = 5, scale: float = 1.5) -> np.ndarray:
    data = data.sort_index()
    data["count"] = data["count"].astype(float)
    counts = data["count"].values
    slopes = np.abs(np.diff(counts))
    moving_avg = np.convolve(slopes, np.ones(wndw) / wndw, mode="same")
    threshold = np.mean(moving_avg) * scale
    background_mask = moving_avg < threshold
    background_mask = np.append(
        background_mask, True
    )  # Ensure the mask has the same length as counts
    background = np.interp(
        np.arange(len(counts)),
        np.arange(len(counts))[background_mask],
        counts[background_mask],
    )
    return background

def format_isotope(isotope):
    match = re.match(r"([a-zA-Z]+)(\d+)", isotope)
    if match:
        element, mass = match.groups()
        return f"$^{{{mass}}}{element.capitalize()}$"
    else:
        return isotope  # fallback if format doesn't match

In [ ]:
plt.rcParams['text.usetex'] = True

def make_subplot(ax, one_meas_processed_measurement, limitation, idx, title):
    meas_500 = one_meas_processed_measurement.loc[one_meas_processed_measurement["energy"] > 0]
    meas_500["background"] = __identify_background(meas_500)
    meas_500["counts_cleaned"] = meas_500["count"] - meas_500["background"]
    ax.plot(meas_500["energy"], meas_500["count"], label="Messung", color=sns.color_palette()[0], alpha=0.5, linewidth=0.5)
    ax.plot(meas_500["energy"], meas_500["background"], label="Hintergrund", color=sns.color_palette()[0], alpha=1, linewidth=0.5, zorder=0)

    colors = sns.color_palette("dark")
    color_idx = 0
    colors_dict = {}
    last_x = 0
    for _, row in meas_500[meas_500["peak"] == True].iterrows():
        x = row["energy"]
        label = row["identified_isotope"]
        if label in colors_dict.keys():
            color = colors_dict[label]
        else:
            color = colors[color_idx]
            colors_dict[label] = color
            color_idx += 1
        if x - last_x < 100 and last_x > 0:
            space = x +20
        else:
            space=0

        last_x = x
        ax.vlines(
            x=x,
            ymin=0,
            ymax=1_000_000,
            color=color,
            alpha=0.5,
            linewidth=0.3,
            linestyle=(0, (5, 5))
        )
            # Plot text slightly above the bottom of the line
        ax.annotate(
            text=label,
            xy=(x, limitation),               # point to peak (start of arrow)
            xytext=(x + space, limitation * 1.15),  # label position (to the right and above)
            textcoords='data',
            fontsize=12,
            color=color,
            # rotation=90,
            rotation=90,
            ha='left',
            va='bottom',
            arrowprops=dict(
                arrowstyle='-',
                connectionstyle='arc,angleA=90,angleB=0,rad=0,armA=10,armB=0',
                color=color,
                linewidth=0.3,
                linestyle=(0, (5, 5)),
                alpha=0.5
            )
        )

    ax.set_title(title, size=12)
    ax.set_ylim(0,limitation)
    ax.set_xlim(0,2000)
    # if idx == 4:
    #     ax.set_xlabel("Energie [keV]", size=14)
    # if idx == 0 or idx == 3:
    #     ax.set_ylabel("Zählwert", size=14)
    ax.tick_params(axis='x', labelsize=12, bottom=True)
    ax.tick_params(axis='y', labelsize=12, left=True)
    ax.grid(False)
    # ax = ax.gca()
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    # ax.spines["left"].set_visible(False)
    # ax.spines["bottom"].set_visible(False)

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(10, 5), sharex=True)
plt.subplots_adjust(hspace=0.5)
good_examples = [0, 5, 234, -10, 18, -5]
axs = axs.flatten()  # now axs is a 1D array
# one_meas_synthetics =gpi.API().synthetics_for_meas([date])

fig.text(0.5, 0.0, "Energie [keV]", size=14, ha='center')
fig.text(0.05, 0.5, "Zählwert", size=14, va='center', rotation='vertical')
limitations = [30000, 60 ,100, 200, 2000 ,1000]
titles = ["(A)", "(B)", "(C)", "(D)", "(E)", "(F)"]
for idx, sample in enumerate(good_examples):
    date = dates[sample]
    limitation = limitations[idx]
    one_meas_processed_measurement = processed_measurements.loc[processed_measurements["datetime"] == date]
    one_meas_processed_measurement["identified_isotope"] = one_meas_processed_measurement["identified_isotope"].apply(format_isotope)
    make_subplot(axs[idx], one_meas_processed_measurement, limitation, idx, titles[idx])
leg = plt.legend(
    loc="upper center",
    bbox_to_anchor=(-0.65, -0.45),
    borderaxespad=0,
    ncol=3,
    frameon=True
)
for line in leg.get_lines():
    line.set_linewidth(3)
plt.savefig("plots/background_meas_example2.pdf", bbox_inches='tight')
# 3406,5
# 54512,64
# 8449,22
# 500

In [ ]:
keys = ppi.API().unique_dates()
re_processed_measurements = ppi.API().re_measurement(keys)

In [ ]:
plt.rcParams['text.usetex'] = True
from scipy.signal import savgol_filter

def make_subplot(ax, one_meas_processed_measurement, limitation, idx, title):
    meas_500 = one_meas_processed_measurement.loc[one_meas_processed_measurement["energy"] > 0]
    meas_500["background"] = savgol_filter(meas_500["count"].to_numpy(), window_length=8, polyorder=1)
    meas_500["counts_cleaned"] = meas_500["count"] - meas_500["background"]
    ax.plot(meas_500["energy"], meas_500["count"], label="Messung", color=sns.color_palette()[0], alpha=1, linewidth=0.5)
    label_idx = 0
    for _, row in meas_500[meas_500["peak"] == True].iterrows():
        x = row["energy"]
        label = row["identified_isotope"]

        last_x = x
        if label_idx == 0:
            ax.vlines(
                x=x,
                ymin=0,
                ymax=1_000_000,
                color="black",
                label="Nuklid",
                alpha=0.5,
                linewidth=0.3,
                linestyle=(0, (5, 5))
            )
        else:
            ax.vlines(
                x=x,
                ymin=0,
                ymax=1_000_000,
                color="black",
                alpha=0.5,
                linewidth=0.3,
                linestyle=(0, (5, 5))
            )
        label_idx += 1
    ax.set_title(title, size=12)
    ax.set_ylim(0,limitation)
    ax.set_xlim(0,2000)
    ax.tick_params(axis='x', labelsize=12, bottom=True)
    ax.tick_params(axis='y', labelsize=12, left=True)
    ax.grid(False)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(10, 5), sharex=True)
plt.subplots_adjust(hspace=0.5)
good_examples = [0, 5, 234, -10, 18, -5]
axs = axs.flatten()  # now axs is a 1D array
# one_meas_synthetics =gpi.API().synthetics_for_meas([date])

titles = ["(A)", "(B)", "(C)", "(D)", "(E)", "(F)"]
fig.text(0.5, 0.0, "Energie [keV]", size=14, ha='center')
fig.text(0.05, 0.5, "Zählwert", size=14, va='center', rotation='vertical')
limitations = [30000, 100 ,100, 200, 2000 ,1000]
for idx, sample in enumerate(good_examples):
    date = dates[sample]
    limitation = limitations[idx]
    one_meas_processed_measurement = re_processed_measurements.loc[re_processed_measurements["datetime"] == date]
    one_meas_processed_measurement["identified_isotope"] = one_meas_processed_measurement["identified_isotope"].apply(format_isotope)
    make_subplot(axs[idx], one_meas_processed_measurement, limitation, idx, titles[idx])
leg = plt.legend(
    loc="upper center",
    bbox_to_anchor=(-0.65, -0.45),
    borderaxespad=0,
    ncol=3,
    frameon=True
)
for line in leg.get_lines():
    line.set_linewidth(3)
plt.savefig("plots/background_meas_example3.pdf", bbox_inches='tight')

In [ ]:
plt.rcParams['text.usetex'] = True

def make_subplot(ax, one_meas_processed_measurement, limitation, idx, title):
    meas_500 = one_meas_processed_measurement.loc[one_meas_processed_measurement["energy"] > 0]
    meas_500["background"] = __identify_background(meas_500)
    meas_500["counts_cleaned"] = meas_500["count"] - meas_500["background"]
    ax.plot(meas_500["energy"], meas_500["count"], label="Synthetische Daten",
            color=sns.color_palette()[0], alpha=1, linewidth=0.5)

    label_idx = 0
    for _, row in meas_500[meas_500["peak"] == True].iterrows():
        x = row["energy"]
        label = row["identified_isotope"]
        if label_idx == 0:
            ax.vlines(
                x=x,
                ymin=0,
                ymax=1_000_000,
                color="black",
                label="Nuklid",
                alpha=0.5,
                linewidth=0.3,
                linestyle=(0, (5, 5))
            )
        else:
            ax.vlines(
                x=x,
                ymin=0,
                ymax=1_000_000,
                color="black",
                alpha=0.5,
                linewidth=0.3,
                linestyle=(0, (5, 5))
            )
        label_idx += 1

    ax.set_title(title, size=12)
    ax.set_ylim(0,limitation)
    ax.set_xlim(0,2000)
    ax.tick_params(axis='x', labelsize=12, bottom=True)
    ax.tick_params(axis='y', labelsize=12, left=True)
    ax.grid(False)
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(10, 5), sharex=True)
plt.subplots_adjust(hspace=0.5)
good_examples = [0, 1, 2, -10, 18, -5]
axs = axs.flatten()  # now axs is a 1D array
# one_meas_synthetics =gpi.API().synthetics_for_meas([date])

titles = ["(A)", "(B)", "(C)", "(D)", "(E)", "(F)"]
fig.text(0.5, 0.0, "Energie [keV]", size=14, ha='center')
fig.text(0.05, 0.5, "Zählwert", size=14, va='center', rotation='vertical')
limitations = [100, 10 ,2000, 4000, 400 ,2000]
for idx, sample in enumerate(good_examples):
    date = synthetic_keys[0:1246][sample]
    limitation = limitations[idx]
    one_meas_processed_measurement = synthetics.loc[synthetics["datetime"] == date]
    one_meas_processed_measurement["identified_isotope"] = one_meas_processed_measurement["identified_isotope"].apply(format_isotope)
    make_subplot(axs[idx], one_meas_processed_measurement, limitation, idx, titles[idx])
handles_labels = [axs[i].get_legend_handles_labels() for i in range(len(axs))]
handles, labels = [], []
for h, l in handles_labels:
    for handle, label in zip(h, l):
        if label not in labels:  # avoid duplicates
            handles.append(handle)
            labels.append(label)
leg = fig.legend(
    handles, labels,
    loc="upper center",
    bbox_to_anchor=(0.5, -0.05),  # or your desired position
    ncol=2,
    frameon=True
)
for line in leg.get_lines():
    line.set_linewidth(3)
plt.savefig("plots/background_meas_example4.pdf", bbox_inches='tight')

In [ ]:
meas_500["count"].max() / 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
np.random.seed(42)
common_value = 0.345
distances = measurement_diffs["energy_diffs"].to_numpy()

# Kandidatenwerte: rund um den Mittelwert
candidates = np.linspace(0.28, 0.38, 100)

# Quadratischer Fehler für jeden Kandidaten
errors = [(val, np.mean((distances - val) ** 2)) for val in candidates]
x_vals, loss_vals = zip(*errors)

# Minimum berechnen
optimal_val = np.mean(distances)
min_loss = np.mean((distances - optimal_val) ** 2)


# Plot
plt.figure(figsize=(6, 3))
plt.plot(x_vals, loss_vals, label="MSE", color="black")
plt.axvline(optimal_val, color="lightgrey", linestyle="--", label=f"$\mu$ = {optimal_val:.5f}")
plt.axhline(min_loss, color="black", linestyle="--", label=f"Minumum MSE = {min_loss:.5f}")
plt.xlabel("Interpolationswert", size=14, labelpad=10)
plt.ylabel("MSE", size=14, labelpad=10)
plt.tick_params(axis='x', labelsize=12, bottom=True)
plt.tick_params(axis='y', labelsize=12, left=True)
plt.grid(False)
plt.legend()
plt.tight_layout()
leg = plt.legend(
    loc="lower center",
    bbox_to_anchor=(0.5, 1.02),
    borderaxespad=0,
    ncol=4,
    frameon=False
)
for line in leg.get_lines():
    line.set_linewidth(3)
plt.tight_layout()
plt.savefig("plots/min_error.pdf")


optimal_val, min_loss
